In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import urllib.request, time, os

In [2]:
def ts(x):
    time.sleep(x)

## 1. 

In [3]:
# //*[@id="query"]
# //*[@id="place-main-section-root"]/section/div/div[6]/a

In [4]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())); ts(2)
driver.get('https://www.google.co.kr/'); ts(3)
driver.get('https://www.naver.com'); ts(2)

driver.find_element(By.ID, 'query').send_keys('관악구 돈가스' + '\n'); ts(3)
driver.find_element(By.XPATH, '//*[@id="place-main-section-root"]/section/div/div[6]/a').click()

driver.switch_to.window(driver.window_handles[1])

# iframe 안으로 들어가기
driver.switch_to.frame('searchIframe')

# iframe 안쪽을 한번 클릭하기
driver.find_element(By.ID, '_pcmap_list_scroll_container').click()

# 로딩된 데이터 개수 확인
res = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')
before_len = len(res)

# 페이지 아래로 내리기
while True:
    # 맨 아래로 스크롤 내린다.
    driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.END)

    # 스크롤 사이 페이지 로딩 시간
    ts(1.5)

    # 스크롤 후 로딩된 데이터 개수 확인
    res = driver.find_elements(By.CSS_SELECTOR, '#_pcmap_list_scroll_container > ul > li')
    after_len = len(res)

    # 로딩된 데이터 개수가 같다면 반복 멈춤
    if before_len == after_len:
        break
    before_len = after_len

In [ ]:
# #_pcmap_list_scroll_container > ul > li:nth-child(1)
# #_pcmap_list_scroll_container > ul > li:nth-child(50)

# #_pcmap_list_scroll_container > ul > li:nth-child(50)

In [5]:
after_len

50

In [5]:
driver.window_handles

['82FCFEFD43638380FCBB4011D298D985', 'DE5E1DAC4932DA21BBA20C8351C26695']

In [ ]:
driver.switch_to.frame()

In [6]:
driver.close()

In [7]:
driver.switch_to.window(driver.window_handles[0])
driver.close()

In [15]:
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.171)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00C92B63+47795]
	(No symbol) [0x00C267E1]
	(No symbol) [0x00B2C7ED]
	(No symbol) [0x00B13495]
	(No symbol) [0x00B7893B]
	(No symbol) [0x00B7CED4]
	(No symbol) [0x00B74C93]
	(No symbol) [0x00B513D7]
	(No symbol) [0x00B5255D]
	GetHandleVerifier [0x00EE6F89+2490073]
	GetHandleVerifier [0x00F2C361+2773681]
	GetHandleVerifier [0x00F26211+2748769]
	GetHandleVerifier [0x00D116B0+566784]
	(No symbol) [0x00C3019A]
	(No symbol) [0x00C2C418]
	(No symbol) [0x00C2C4FB]
	(No symbol) [0x00C1F317]
	BaseThreadInitThunk [0x75D000C9+25]
	RtlGetAppContainerNamedObjectPath [0x77EA7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77EA7AEE+238]
